In [8]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

#import data file
df = pd.read_csv("Data/Export/Export.csv")
df2 = pd.read_csv("Data/transcript.csv")

#drop duplicate IDs
df = df.drop_duplicates(subset=(['ID']))

#drop instances without a math, english, and science class
df = df.dropna(subset=(['math9', 'math10', 'eng9', 'eng10', 'bio9', 'phys10']))

#replace double_math nan with 0 and yes with 1
df['Double_Math'] = df['Double_Math'].fillna(0)
df['Double_Math'] = df['Double_Math'].replace('yes', 1)

#show first few rows
#df.head()

In [10]:
df.columns

Index(['ID', 'math9', 'math9_score', 'math10', 'math10_score', 'eng9',
       'eng9_score', 'eng10', 'eng10_score', 'hist9', 'hist9_score', 'bio9',
       'bio_score', 'phys10', 'phys10_score', 'chem11', 'chem11_score',
       'Gender', 'Counselor', 'ELL_Status', 'Double_Math', 'FRPL_Status',
       'Grade_Level', 'Home_Language', 'Date_of_Birth', 'Race_Ethnicity',
       'Transfer_Student', 'Special_Education_Status', 'schoolName'],
      dtype='object')

In [16]:
#count number of unique values for gender
df.Gender.unique #returns mostly NaN
df.Counselor.unique #returns mostly NaN
df.FRPL_Status.unique #mostly NaN 
df.Race_Ethnicity.unique #mostly NaN

<bound method Series.unique of 0       Hispanic
6       Hispanic
15      Hispanic
16           NaN
25           NaN
          ...   
2909         NaN
2912         NaN
2913         NaN
2915         NaN
2959         NaN
Name: Race_Ethnicity, Length: 972, dtype: object>